In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import json
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("MENTAL HEALTH TEXT CLASSIFICATION")
print("="*70)

# ============= LOAD MODEL =============
MODEL_PATH = './models/mental_health_roberta'

print(f"\n[1] Loading model from: {MODEL_PATH}")

try:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    
    # Load label mapping
    with open(f'{MODEL_PATH}/label_mapping.json', 'r') as f:
        label_mapping = json.load(f)
    
    # Reverse mapping: {0: 'Normal', 1: 'Anxiety', ...}
    id_to_label = {v: k for k, v in label_mapping.items()}
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    
    num_params = sum(p.numel() for p in model.parameters())
    
    print(f"+ Model loaded successfully")
    print(f"   Parameters: {num_params / 1e6:.1f}M")
    print(f"   Device: {device}")
    print(f"   Classes: {list(label_mapping.keys())}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise



MENTAL HEALTH TEXT CLASSIFICATION

[1] Loading model from: ./models/mental_health_roberta


Loading weights: 100%|██████████| 201/201 [00:00<00:00, 307.33it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              


+ Model loaded successfully
   Parameters: 124.7M
   Device: cuda
   Classes: ['Normal', 'Anxiety', 'Depression', 'Stress', 'Suicidal', 'Bipolar', 'Personality disorder']


In [12]:
# ============= CLASSIFICATION FUNCTIONS =============

def classify_batch(texts):
    inputs = tokenizer(
        texts,
        return_tensors='pt',
        truncation=True,
        max_length=512,
        padding=True
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).cpu()
        predicted_ids = torch.argmax(probs, dim=1)
    
    results = []
    for i, text in enumerate(texts):
        predicted_id = predicted_ids[i].item()
        confidence = probs[i][predicted_id].item()
        
        results.append({
            'Text': text[:80] + '...' if len(text) > 80 else text,
            'Predicted_Class': id_to_label[predicted_id],
            'Confidence': f"{confidence:.2%}",
            'Confidence_Score': confidence
        })
    
    df = pd.DataFrame(results)
    
    print("\n" + "="*70)
    print(f"+ BATCH CLASSIFICATION RESULTS - {len(texts)} texts")
    print("="*70)
    
    # Summary statistics
    class_counts = df['Predicted_Class'].value_counts()
    print("\n+ Class Distribution:")
    for class_name, count in class_counts.items():
        print(f"  {class_name:<25} {count:>3} ({count/len(texts)*100:.1f}%)")
    
    avg_confidence = df['Confidence_Score'].mean()
    print(f"\n+ Average Confidence: {avg_confidence:.2%}")
    
    return df



In [13]:
texts = [
    "Life doesn't seem to have any meaning, one day feels sunny while the next feels dark, my mood swings are extreme, I go from euphoric highs to crushing lows",
    "I have suicidal thoughts",
    "God i have test tomorrow and i can't focus on studying, should i play my video game instead",
    "Some days I’m unstoppable — 3 hours of sleep, big plans, huge confidence, talking too fast, spending too much. Other days I can’t answer a text. I can’t shower. I can’t think clearly. The hardest part isn’t the highs or the lows. It’s the unpredictability. Medication helps. Therapy helps. Structure helps. But it’s still work every day."
]
df = classify_batch(texts)
display(df)


+ BATCH CLASSIFICATION RESULTS - 4 texts

+ Class Distribution:
  Depression                  1 (25.0%)
  Suicidal                    1 (25.0%)
  Normal                      1 (25.0%)
  Stress                      1 (25.0%)

+ Average Confidence: 89.12%


,Text,Predicted_Class,Confidence,Confidence_Score
0,"Life doesn't seem to have any meaning, one day...",Depression,63.13%,0.631349
1,I have suicidal thoughts,Suicidal,95.77%,0.957665
2,God i have test tomorrow and i can't focus on ...,Normal,98.17%,0.981666
3,"Some days I’m unstoppable — 3 hours of sleep, ...",Stress,99.42%,0.994204
